In [17]:
import numpy as np
import pandas as pd
%matplotlib inline

import random
import pprint

from RandomForest_decision_tree_funct import decision_tree_algorithm, decision_tree_predictions
from RandomForest_helper_funct import train_test_split, calculate_accuracy


In [25]:
df = pd.read_csv("Features2.csv")
#df.head()

In [26]:
train_df, test_df = train_test_split(df, test_size = 0.2)
train_df.head()

,contour_points,amount_contours,rect_area,hull_area,approximation_area,contour_perimeters,corners,harris_corners,ratio_wide_length,contour_length_area_ratio,contour_length_rect_area_ratio,contour_length_hull_area_ratio,contour_rect_length_ratio,contour_hull_length_ratio,extent,solidity,hull_rectangle_ratio,labels
0,676,3,20729.545245,14977.0,10152.5,789.494509,53,901,3.399329,0.077310,0.038085,0.052714,1.149038,1.318301,0.492630,0.681845,0.722495,0
1,1725,3,65025.000000,65025.0,48154.5,1847.192994,92,1582,1.000000,0.038508,0.028407,0.028407,1.810974,1.810974,0.737693,0.737693,1.000000,0
3,670,2,19416.432575,15594.0,12276.5,727.989895,59,970,0.336447,0.059283,0.037493,0.046684,1.133750,1.289037,0.632454,0.787482,0.803134,0
4,750,2,23913.438783,19207.5,14916.5,905.744290,71,950,3.920758,0.060291,0.037876,0.047156,1.178440,1.319058,0.628224,0.782142,0.803209,0
5,1656,1,65025.000000,65025.0,51777.0,1792.690468,59,715,1.000000,0.034637,0.027569,0.027569,1.757540,1.757540,0.795940,0.795940,1.000000,0


In [27]:
def bootstrapping(train_df, n_bootstrap):
    bootstrap_indices = np.random.randint(low = 0,  high = len(train_df), size = n_bootstrap)
    df_bootstrapped = train_df.iloc[bootstrap_indices]

    return df_bootstrapped

In [28]:
def random_forest_algorithm(train_df, n_trees, n_bootstrap, n_features, dt_max_depth):
    forest = []
    for i in range(n_trees):
        df_bootstrapped = bootstrapping(train_df, n_bootstrap)
        tree = decision_tree_algorithm(df_bootstrapped, max_depth=dt_max_depth, random_subspace=n_features)
        forest.append(tree)

    return forest

In [36]:
forest = random_forest_algorithm(train_df, n_trees = 4, n_bootstrap= 800, n_features=4, dt_max_depth=4)
pprint.pprint(forest[0])
pprint.pprint(forest[1])
pprint.pprint(forest[2])
pprint.pprint(forest[3])

{'contour_length_hull_area_ratio <= 0.06867133629579064': [{'contour_hull_length_ratio <= 1.1173568617230547': [1.0,
                                                                                                                {'harris_corners <= 127.5': [1.0,
                                                                                                                                             0.0]}]},
                                                           1.0]}
{'solidity <= 0.653109764860431': [1.0,
                                   {'corners <= 51.0': [{'contour_perimeters <= 595.0264731645584': [{'hull_rectangle_ratio <= 0.8574161427791327': [0.0,
                                                                                                                                                     1.0]},
                                                                                                     1.0]},
                                                        0.0]}]}


In [30]:
def random_forest_predictions(test_df, forest):
    df_predictions = {}
    for i in range(len(forest)):
        column_name = "tree_{}".format(i)
        predictions = decision_tree_predictions(test_df, tree=forest[i])
        df_predictions[column_name] = predictions

    df_predictions = pd.DataFrame(df_predictions)
    random_forest_predictions = df_predictions.mode(axis=1)[0]

    return random_forest_predictions

In [32]:
forest = random_forest_algorithm(train_df, n_trees=4, n_bootstrap=800, n_features=2, dt_max_depth=4)
predictions = random_forest_predictions(test_df, forest)
accuracy = calculate_accuracy(predictions, test_df.labels)

print("Accuracy = {}".format(accuracy))



Accuracy = 0.75
